# Exercícios – Structured Streaming

#### Questão 1, exercício completo utilizando o pyspark shell.

#### Questão 2, Ler os arquivos csv “hdfs://namenode:8020/user/thiago/data/iris/*.data” em modo streaming com o seguinte schema:

* sepal_length float
* sepal_width float
* petal_length float
* petal_width float
* class string

In [ ]:
# entendendo o arquivo
!hdfs dfs -ls /user/thiago/data/iris
!hdfs dfs -cat /user/thiago/data/iris/iris.data

# constata-se que o arquivo bezdekIris.data possui 150 linhas
# constata-se que o arquivo iris.data possui 150 linhas

Found 3 items
-rw-r--r--   3 root supergroup       4551 2022-04-08 03:38 /user/thiago/data/iris/bezdekIris.data
-rw-r--r--   3 root supergroup       4551 2022-04-08 03:38 /user/thiago/data/iris/iris.data
-rw-r--r--   3 root supergroup       2998 2022-04-08 03:38 /user/thiago/data/iris/iris.names
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa
5.4,3.7,1.5,0.2,Iris-setosa
4.8,3.4,1.6,0.2,Iris-setosa
4.8,3.0,1.4,0.1,Iris-setosa
4.3,3.0,1.1,0.1,Iris-setosa
5.8,4.0,1.2,0.2,Iris-setosa
5.7,4.4,1.5,0.4,Iris-setosa
5.4,3.9,1.3,0.4,Iris-setosa
5.1,3.5,1.4,0.3,Iris-setosa
5.7,3.8,1.7,0.3,Iris-setosa
5.1,3.8,1.5,0.3,Iris-setosa
5.4,3.4,1.7,0.2,Iris-setosa
5.1,3.7,1.5,0.4,Iris-setosa
4.6,3.6,1.0,0.2,Iris-setosa
5.1,3.3,1.7,0.5,Iris-setosa
4.8,3.4,1.9,0.2,Iris-setosa
5.0

In [ ]:
from pyspark.sql.types import StructType

In [ ]:
# sempre que estiver trabalhando com 'Stream' é necessário fazer o 'Schema'
iris_schema = StructType()\
    .add("sepal_lenght", "float")\
    .add("sepal_width", "float")\
    .add("petal_lenght", "float")\
    .add("petal_width", "float")\
    .add("class", "string")

In [ ]:
print(iris_schema)

StructType(List(StructField(sepal_lenght,FloatType,true),StructField(sepal_width,FloatType,true),StructField(petal_lenght,FloatType,true),StructField(petal_width,FloatType,true),StructField(class,StringType,true)))


In [ ]:
# analisando Schema
iris = spark.read.schema(iris_schema).csv("/user/thiago/data/iris/*.data").show(10)
iris = spark.read.schema(iris_schema).csv("/user/thiago/data/iris/*.data").printSchema()

+------------+-----------+------------+-----------+-----------+
|sepal_lenght|sepal_width|petal_lenght|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 10 rows

root
 |-- sepal_lenght: float (nullable = true)
 |-- sepal_width: float (nulla

In [ ]:
# leitura em Stream
iris = spark.readStream.schema(iris_schema).csv("/user/thiago/data/iris/*.data")

#### Salvar os dados no diretório “hdfs://namenode:8020/user/thiago/stream_iris/path” e o checkpoint em “hdfs://namenode:8020/user/thiago/stream_iris/check”

In [ ]:
iris_saida = iris.writeStream.format("csv")\
    .option("checkpointLocation", "/user/thiago/stream_iris/check")\
    .option("path", "/user/rodrigo/stream_iris/path")\
    .start()

In [ ]:
iris_saida.status

{'message': 'Getting offsets from FileStreamSource[hdfs://namenode:8020/user/thiago/data/iris/*.data]',
 'isDataAvailable': False,
 'isTriggerActive': True}

#### Verificar a saida no hdfs e entender como os dados foram salvos

In [ ]:
!hdfs dfs -ls /user/thiago/stream_iris/check

Found 4 items
drwxr-xr-x   - root supergroup          0 2022-04-19 04:19 /user/thiago/stream_iris/check/commits
-rw-r--r--   3 root supergroup         45 2022-04-19 04:19 /user/thiago/stream_iris/check/metadata
drwxr-xr-x   - root supergroup          0 2022-04-19 04:19 /user/thiago/stream_iris/check/offsets
drwxr-xr-x   - root supergroup          0 2022-04-19 04:19 /user/thiago/stream_iris/check/sources


In [ ]:
!hdfs dfs -ls /user/rodrigo/stream_iris/path

Found 3 items
drwxr-xr-x   - root supergroup          0 2022-04-19 04:17 /user/rodrigo/stream_iris/path/_spark_metadata
-rw-r--r--   2 root supergroup       4550 2022-04-19 04:17 /user/rodrigo/stream_iris/path/part-00000-c72cd534-e14e-4dfb-be34-6d616eec0d2f-c000.csv
-rw-r--r--   2 root supergroup       4550 2022-04-19 04:17 /user/rodrigo/stream_iris/path/part-00001-c9ec8490-261d-451b-9b3e-66fa6f05de8a-c000.csv
